In [118]:
using ITensors, ITensorMPS
using LinearAlgebra
using Plots
using HDF5

N = 18

f4 = h5open("Basis states Final/MPO_$N.h5","r")
W1 = read(f4,"W",MPO)
close(f4)

# Assuming `siteinds(W1)` gives the indices of the MPO
all_sites = siteinds(W1)

# Consider only physical sites
s = [pair[2] for pair in all_sites]

# println(s)

18-element Vector{Index{Int64}}:
 (dim=2|id=261|"Qubit,Site,n=1")
 (dim=2|id=932|"Qubit,Site,n=2")
 (dim=2|id=285|"Qubit,Site,n=3")
 (dim=2|id=83|"Qubit,Site,n=4")
 (dim=2|id=1|"Qubit,Site,n=5")
 (dim=2|id=915|"Qubit,Site,n=6")
 (dim=2|id=348|"Qubit,Site,n=7")
 (dim=2|id=346|"Qubit,Site,n=8")
 (dim=2|id=523|"Qubit,Site,n=9")
 (dim=2|id=169|"Qubit,Site,n=10")
 (dim=2|id=73|"Qubit,Site,n=11")
 (dim=2|id=159|"Qubit,Site,n=12")
 (dim=2|id=270|"Qubit,Site,n=13")
 (dim=2|id=435|"Qubit,Site,n=14")
 (dim=2|id=959|"Qubit,Site,n=15")
 (dim=2|id=379|"Qubit,Site,n=16")
 (dim=2|id=587|"Qubit,Site,n=17")
 (dim=2|id=520|"Qubit,Site,n=18")

In [119]:
# Taking the input 

x = range(0, stop=2π, length=2^(N))
input_array = cos.(x)


array = input_array / norm(input_array) # Input
ITensors.disable_warn_order()

cutoff1 = 1E-24
maxdim1 = 10


T = ITensor(array,s)

ψ = MPS(T,s;cutoff=cutoff1,maxdim=maxdim1)

orthogonalize!(ψ, 1) # Orthogonalize Psi 

start = time()

result = contract(W1, ψ)
end_time = time()
time1 = end_time - start

println(time1)

0.00415492057800293


In [120]:
using FFTW

start2 = time()
x2 = fft(array)
end2 = time()

time2 = end2 - start2
println(time2)

0.016412973403930664


# Check Output 

In [121]:

inner_products = []
MPS2 = []

for i in 1:2^3
    f2 = h5open("Basis states Final/MPS_$N/MPS_create_$i.h5","r")
    mps1 = read(f2,"M",MPS)
    push!(MPS2,mps1)
    close(f2)
end

for i in 1:(2^3)
    push!(inner_products, inner(MPS2[i], result))
    print(inner(MPS2[i], result)*2^(N/2))
    println()
end


0.002762130599815013 + 2.979714937090989e-14im
362.0386719348623 + 0.0043387527501762896im
-0.0009207195593787514 - 2.206799722679766e-8im
-0.0003452692844384114 - 1.2413259059515352e-8im
-0.00017297471297654526 - 8.291829373187094e-9im
-0.0001150898159370587 - 6.895841306200925e-9im
-9.852659508225201e-5 - 7.0845299319945645e-9im
-5.754487369348539e-5 - 4.81798097840954e-9im


In [122]:
for i in 1:2^3
    println(x2[i])
end

0.0027621305956405428 + 0.0im
362.03867193482245 + 0.004338752869069623im
-0.000920719564220234 - 2.2068220727816898e-8im
-0.00034526928762401417 - 1.2413352371348043e-8im
-0.00018414353798833466 - 8.827270340894452e-9im
-0.0001150896891976244 - 6.896297819640831e-9im
-7.89186360494099e-5 - 5.674672125442224e-9im
-5.7544835317219315e-5 - 4.827417004817351e-9im


# Code to generate basis states to measure

In [112]:
# function generate_basis_states(n)
#     basis_states = []
#     for i in 0:(2^n - 1)
#         binary_str = bitstring(i)[end-n+1:end]  # Get last n bits of bitstring
#         push!(basis_states, collect(binary_str))  # Collect splits string into array of characters
#     end
#     return basis_states
# end

# MPS_states = []
# basis_states = generate_basis_states(N)

# for i in 1:(2^N)
#     push!(MPS_states, MPS(s,string.(basis_states[i])))
# end

# using HDF5
# f = 0 

# for (i, mps) in enumerate(MPS_states)
#     f = h5open("Basis states Final/MPS_$N/MPS_create_$i.h5","w")
#     write(f,"M", mps)
# end
# close(f)